In [1]:
import os
os.chdir(os.path.join(os.getcwd(), '..'))

In [2]:
import yaml
from models.bendr import EncoderConv, ContextNetwork
import torch
import yaml
import numpy as np

with open('configs/cluster_config_bendr.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

encoder_sd = torch.load('/home/ps5359/checkpoints/encoder_2024-10-08T00:17:31.296499.pt', map_location=torch.device('cpu') )
context_network_sd = torch.load('/home/ps5359/checkpoints/context_network_2024-10-08T00:17:31.296499.pt', map_location=torch.device('cpu') )


encoder = EncoderConv(**cfg['encoder'])
context_network = ContextNetwork(**cfg['context_network'])

encoder.load_state_dict(encoder_sd)
context_network.load_state_dict(context_network_sd)

/ext3/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>

In [3]:
from dataset.labeled_dataset import EEGLabeledDataset
from torch.utils.data import DataLoader

dataset = EEGLabeledDataset(
    data_path='/home/ps5359/data/full_clean/h5',
    cache_processed_path='/vast/ps5359/full_clean/cache/',
    train_length=73728, dataset_mode="full",
    rebuild_cache=False, limit=None,
    target_config={
    "user_id": True,
    "activity": False, },
    clipped_threshold=100
)
loader = DataLoader(dataset, 4, shuffle=False, drop_last=False)
next(iter(loader))['data'].shape


torch.Size([4, 4, 73728])

In [4]:
batch = next(iter(loader))
batch = encoder(batch)
batch = context_network(batch)
batch

/ext3/miniforge3/lib/python3.12/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


{'data': tensor([[[-1.3341, -1.2536, -0.9562,  ..., -0.4517, -0.6568, -0.3976],
          [ 0.2220,  0.4329,  0.6511,  ...,  0.0171,  0.0758,  0.0437],
          [-0.0994, -0.3084, -0.4801,  ...,  0.0535, -0.2214, -0.4292],
          [ 0.1419,  0.1302,  0.0281,  ...,  0.0217, -0.1909, -0.1806]],
 
         [[ 0.1890,  0.7485,  1.1593,  ..., -0.9186, -1.0835, -1.0746],
          [ 0.0065,  0.0361,  0.1696,  ...,  1.1903,  1.2647,  1.2332],
          [-0.3983, -0.3128, -0.0671,  ..., -0.6552, -0.6781, -0.6625],
          [-0.0156,  0.1706,  0.4004,  ..., -1.1468, -1.1450, -1.1531]],
 
         [[-0.8618, -0.4145,  0.0232,  ..., -0.5427, -0.4618, -0.3440],
          [ 1.2871,  1.3324,  1.3286,  ..., -0.2308, -0.2620, -0.3322],
          [-0.6414, -0.5434, -0.3823,  ..., -0.0653, -0.0803, -0.0248],
          [-1.1474, -0.9334, -0.5973,  ..., -0.2457, -0.3303, -0.2317]],
 
         [[-0.1153,  0.2610,  0.5851,  ..., -0.6944, -0.7370, -0.7869],
          [-0.3611, -0.3088, -0.2901,  ..., -0.

In [7]:
batch['context_vectors'].view(4 * 768, 512)

tensor([[-1.5025e+01,  5.6398e+00, -4.2979e+00,  ...,  1.3047e+01,
          1.2006e+00,  2.1339e+00],
        [ 8.2460e+00,  1.4000e+00, -1.6855e+00,  ...,  3.5975e+00,
          1.8560e+01, -1.0197e+01],
        [-2.0269e+01,  7.6585e-02, -1.6518e+01,  ...,  1.5375e+01,
          2.6788e+00, -1.5532e+01],
        ...,
        [-1.3225e+00,  1.0473e+00, -8.3555e-01,  ...,  1.6862e+00,
          1.5805e-02,  1.0679e+00],
        [-6.7184e-01,  2.2891e+00, -1.3661e+00,  ...,  1.2133e+00,
          2.5799e-01,  3.3633e-01],
        [-1.3405e-02,  1.1261e+00, -1.2121e+00,  ...,  1.9352e+00,
          2.0001e-01,  1.1220e+00]], grad_fn=<ViewBackward0>)